In [13]:
pip install nsepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
from nsepy import get_history
from datetime import datetime, timedelta
import time
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Define the symbols and expiry dates
symbols = [
    "BANKNIFTY", "CNX100", "CNXINFRA", "CNXIT", "CNXPSE", "DEFTY", "DJIA", 
    "FINNIFTY", "FTSE100", "JUNIOR", "MIDCPNIFTY", "MINIFTY", "NFTYMCAP50", 
    "NIFTY", "NIFTYCPSE", "NIFTYINFRA", "NIFTYIT", "NIFTYMID50", "NIFTYNXT50", 
    "NIFTYPSE", "S&P500"
]

# Define function to get the last Thursday of a given month and year
def get_last_thursday(year, month):
    last_day = datetime(year, month, 1) + timedelta(days=32)
    last_day = last_day.replace(day=1) - timedelta(days=1)
    while last_day.weekday() != 3:  # 3 corresponds to Thursday
        last_day -= timedelta(days=1)
    return last_day

# Generate expiry dates for the last month
today = datetime.now()
months_to_fetch = 3  # Fetch data for the last 3 months

expiry_dates = []
for i in range(months_to_fetch):
    month = (today.month - i) % 12 or 12
    year = today.year - ((today.month - i - 1) // 12)
    expiry_dates.append(get_last_thursday(year, month))

# Retry settings
retry_strategy = Retry(
    total=5,
    status_forcelist=[429, 500, 502, 503, 504],
    method_whitelist=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session = requests.Session()
session.mount("http://", adapter)
session.mount("https://", adapter)

# Function to fetch data
def fetch_nse_data(symbol, expiry, strike_price):
    start_date = expiry.replace(day=1)
    end_date = expiry

    try:
        data = get_history(symbol=symbol, start=start_date, end=end_date, index=True, option_type='CE', expiry_date=expiry, strike_price=strike_price, session=session)
        
        if not data.empty:
            filename = f'nse_data_{symbol}_{expiry.strftime("%Y-%m-%d")}.csv'
            data.to_csv(filename, index=True)
            print(f"Data fetched and saved successfully for {symbol} with expiry {expiry.strftime('%Y-%m-%d')}!")
        else:
            print(f"No data found for {symbol} with expiry {expiry.strftime('%Y-%m-%d')}.")
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {symbol} with expiry {expiry.strftime('%Y-%m-%d')}: {e}")
    except Exception as e:
        print(f"An error occurred for {symbol} with expiry {expiry.strftime('%Y-%m-%d')}: {e}")

# Function to fetch data for the provided expiry dates
def fetch_data_for_provided_dates():
    for expiry in expiry_dates:
        for symbol in symbols:
            # Adjust the strike price as per the requirement. Here, an example value is given.
            strike_price = 15000  # Example strike price
            fetch_nse_data(symbol, expiry, strike_price)

# Execute the script to fetch data for the provided expiry dates
fetch_data_for_provided_dates()


An error occurred for BANKNIFTY with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for CNX100 with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for CNXINFRA with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for CNXIT with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for CNXPSE with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for DEFTY with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for DJIA with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for FINNIFTY with expiry 2024-05-30: get_history() got an unexpected keyword argument 'session'
An error occurred for FTSE100 with expiry 2024-05-30: get_history() got an unexpected keyword argumen

C:\Users\Prathmesh\AppData\Local\Temp\ipykernel_29120\383949786.py:36: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  retry_strategy = Retry(


In [17]:
import requests
from nsepy import get_history
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime

def fetch_nse_data(symbol, expiry, strike_price):
    start_date = expiry.replace(day=1)
    end_date = expiry
    
    # Create a session and configure retries
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    # Fetch data using nsepy without passing the session
    try:
        data = get_history(
            symbol=symbol,
            start=start_date,
            end=end_date,
            index=True,
            option_type='CE',
            expiry_date=expiry,
            strike_price=strike_price
        )
        
        if not data.empty:
            filename = f'nse_data_{symbol}_{expiry.strftime("%Y-%m-%d")}.csv'
            data.to_csv(filename)
            print(f'Data saved to {filename}')
        else:
            print(f'No data found for {symbol} with expiry {expiry}')
    except Exception as e:
        print(f'An error occurred for {symbol} with expiry {expiry}: {e}')

def fetch_data_for_provided_dates():
    symbols = ['BANKNIFTY', 'CNX100', 'CNXINFRA', 'CNXIT', 'CNXPSE', 'DEFTY', 'DJIA', 'FINNIFTY', 'FTSE100', 'JUNIOR', 'MIDCPNIFTY', 'MINIFTY', 'NFTYMCAP50', 'NIFTY', 'NIFTYCPSE', 'NIFTYINFRA', 'NIFTYIT', 'NIFTYMID50', 'NIFTYNXT50', 'NIFTYPSE', 'S&P500']
    expiry_dates = [
        datetime(2024, 5, 30),
        datetime(2024, 4, 25),
        datetime(2024, 3, 28)
    ]
    
    for expiry in expiry_dates:
        for symbol in symbols:
            strike_price = 15000  # Example strike price
            fetch_nse_data(symbol, expiry, strike_price)

# Execute the script to fetch data for the provided expiry dates
fetch_data_for_provided_dates()


An error occurred for BANKNIFTY with expiry 2024-05-30 00:00:00: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred for CNX100 with expiry 2024-05-30 00:00:00: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred for CNXINFRA with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for CNXIT with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for CNXPSE with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for DEFTY with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for DJIA with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for FINNIFTY with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for FTSE100 with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for JUNIOR with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An err

In [18]:
import requests
from nsepy import get_history
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime

def fetch_nse_data(symbol, expiry, strike_price):
    start_date = expiry.replace(day=1)
    end_date = expiry
    
    # Create a session and configure retries
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    # Fetch data using nsepy without passing the session
    try:
        data = get_history(
            symbol=symbol,
            start=start_date,
            end=end_date,
            index=True,
            option_type='CE',
            expiry_date=expiry,
            strike_price=strike_price
        )
        
        if not data.empty:
            filename = f'nse_data_{symbol}_{expiry.strftime("%Y-%m-%d")}.csv'
            data.to_csv(filename)
            print(f'Data saved to {filename}')
        else:
            print(f'No data found for {symbol} with expiry {expiry}')
    except Exception as e:
        print(f'An error occurred for {symbol} with expiry {expiry}: {e}')

def fetch_data_for_provided_dates():
    symbols = ['BANKNIFTY', 'CNX100', 'CNXINFRA', 'CNXIT', 'CNXPSE', 'DEFTY', 'DJIA', 'FINNIFTY', 'FTSE100', 'JUNIOR', 'MIDCPNIFTY', 'MINIFTY', 'NFTYMCAP50', 'NIFTY', 'NIFTYCPSE', 'NIFTYINFRA', 'NIFTYIT', 'NIFTYMID50', 'NIFTYNXT50', 'NIFTYPSE', 'S&P500']
    expiry_dates = [
        datetime(2024, 5, 30),
        datetime(2024, 4, 25),
        datetime(2024, 3, 28)
    ]
    
    for expiry in expiry_dates:
        for symbol in symbols:
            strike_price = 15000  # Example strike price
            fetch_nse_data(symbol, expiry, strike_price)

# Execute the script to fetch data for the provided expiry dates
fetch_data_for_provided_dates()


An error occurred for BANKNIFTY with expiry 2024-05-30 00:00:00: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
An error occurred for CNX100 with expiry 2024-05-30 00:00:00: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred for CNXINFRA with expiry 2024-05-30 00:00:00: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred for CNXIT with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for CNXPSE with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for DEFTY with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for DJIA with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for FINNIFTY with expiry 2024-05-30 00:00:00: Exceeded 30 redirects.
An error occurred for FTSE100 with expiry 2024-05-30 00:00:00: 